# Breast Cancer Data Analysis and Predictions

###  Breast Cancer

Breast cancer is cancer that forms in the cells of the breasts.<p>It arises in the lining cells (epithelium) of the ducts (85%) or lobules (15%) in the glandular tissue of the breast. Initially, the cancerous growth is confined to the duct or lobule (“in situ”) where it generally causes no symptoms and has minimal potential for spread (metastasis).<br>
    
Most types of breast cancer are easy to diagnose by microscopic analysis of a sample - or biopsy - of the affected area of the breast. Also, there are types of breast cancer that require specialized lab exams.

# Website for data
https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data

# 1. Exploratory Data Analysis

## 1.1 Understanding the data

In [ ]:
pip install pandas

In [ ]:
# importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [ ]:
# reading data into the dataframe
df = pd.read_csv('/Users/riteshkumar/Downloads/ML/Breast cancer prediction/data.csv')

In [ ]:
# displaying first five rows
df.head()

In [ ]:
# shape of the dataframe
df.shape

In [ ]:
# concise summary of dataframe
df.info()

In [ ]:
# column names
df.columns

In [ ]:
# checking for null values
df.isnull().sum()

The whole column 'Unamed: 32' has NaN values.

In [ ]:
# dropping 'Unnamed: 32' column.
df.drop("Unnamed: 32", axis=1, inplace=True)

In [ ]:
# dropping id column
df.drop('id',axis=1, inplace=True)

In [ ]:
# descriptive statistics of data
df.describe()

## 1.2. Data Visualizations

In [ ]:
# countplot
plt.figure(figsize = (8,7))
sns.countplot(x="diagnosis", data=df, palette='magma')

In [ ]:
# Only select numeric columns
plt.figure(figsize=(20,18))
sns.heatmap(df.select_dtypes(include=['float64', 'int64']).corr(), annot=True, linewidths=.5, cmap="Purples")


From the heatmap, we can observe from the heatmaps that there are many negative correlations in this dataset.

In [ ]:
df.columns

The mean, standard error and "worst" or largest (mean of the three
largest values) of these features were computed for each image,
resulting in 30 features. For instance, field 3 is Mean Radius, field
13 is Radius SE, field 23 is Worst Radius.

In [ ]:
# Getting Mean Columns with diagnosis
m_col = ['diagnosis','radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']

# Getting Se Columns with diagnosis
s_col = ['diagnosis','radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se']

# Getting Worst column with diagnosis
w_col = ['diagnosis','radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']

### For Mean Columns

In [ ]:
# pairplot for mean columns
sns.pairplot(df[m_col],hue = 'diagnosis', palette='Blues')

### For SE columns

In [ ]:
# pairplot for se columns
sns.pairplot(df[s_col],hue = 'diagnosis', palette='Greens')

### For Worst columns

In [ ]:
# pairplot for worst columns
sns.pairplot(df[w_col],hue = 'diagnosis', palette='Oranges')

# 2. Data Preprocessing and Building Models

## 2.1 Data Preprocessing

In [ ]:
# counts of unique rows in the 'diagnosis' column
df['diagnosis'].value_counts()

In [ ]:
# mapping categorical values to numerical values
df['diagnosis']=df['diagnosis'].map({'B':0,'M':1})

In [ ]:
df['diagnosis'].value_counts()

## 2.2 Splitting the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
                df.drop('diagnosis', axis=1),
                df['diagnosis'],
                test_size=0.2,
                random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance.(Unit variance means dividing all the values by the standard deviation.)

## 2.3 Classification Models

### 2.3.1 Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions1 = logreg.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions1))
print('\n')
print(classification_report(y_test, predictions1))

In [ ]:
from sklearn.metrics import accuracy_score

logreg_acc = accuracy_score(y_test, predictions1)
print("Accuracy of the Logistic Regression Model is: ", logreg_acc)

### 2.3.2 K Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# to find which value shows the lowest mean error
error_rate = []

for i in range(1,42):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(1,42), error_rate, color='purple', linestyle="--",
         marker='o', markersize=10, markerfacecolor='b')
plt.title('Error_Rate vs K-value')
plt.show()

From this graph, K value of 9,34,35,36,40 and 41 seem to show the lowest mean error. So using one of these values

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)
predictions2 = knn.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, predictions2))
print("\n")
print(classification_report(y_test, predictions2))

In [ ]:
knn_model_acc = accuracy_score(y_test, predictions2)
print("Accuracy of K Neighbors Classifier Model is: ", knn_model_acc)

### 2.3.3 Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=300)
rfc.fit(X_train, y_train)
predictions4 = rfc.predict(X_test)

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions4))
print("\n")
print(classification_report(y_test, predictions4))

In [ ]:
rfc_acc = accuracy_score(y_test, predictions4)
print("Accuracy of Random Forests Model is: ", rfc_acc)

### 2.3.4 Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

svc_model = SVC(kernel="rbf")
svc_model.fit(X_train, y_train)
predictions5 = svc_model.predict(X_test)

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions5))
print("\n")
print(classification_report(y_test, predictions5))

In [ ]:
svm_acc = accuracy_score(y_test, predictions5)
print("Accuracy of SVM model is: ", svm_acc)

In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler

# After fitting the scaler and training the model
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save scaler and model
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(logreg, 'breast_cancer_model.pkl')  # or any other model you want to use


# 3. Final Results

In [ ]:
print(logreg_acc)
print(knn_model_acc)
print(rfc_acc)
print(svm_acc)

The accuracy of Logistic Regression Model is **98.24%**<br>
The accuracy of KNN model is **96.49%**<br>
The accuracy of Random Forest Model is **96.49%**<br>
The accuracy of SVM Model is **98.24%**<br>

In [ ]:
plt.figure(figsize=(12,6))
model_acc = [logreg_acc, knn_model_acc, rfc_acc, svm_acc]
model_name = ['LogisticRegression', 'KNN', 'RandomForests', 'SVM']
sns.barplot(x= model_acc, y=model_name, palette='magma')

Logistic Regression model and SVM model gave best performance with an accuracy of **98.24%**

In [ ]:
# 1. Save the trained model
import joblib
joblib.dump(logreg, 'breast_cancer_model.pkl')


In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import joblib

# Load model and scaler
model = joblib.load('breast_cancer_model.pkl')
scaler = joblib.load('scaler.pkl')

# Tkinter window setup
root = tk.Tk()
root.title("Breast Cancer Prediction")
root.geometry("600x750")
root.configure(bg="black")

# Center frame
center_frame = tk.Frame(root, bg="black")
center_frame.place(relx=0.5, rely=0.5, anchor="center")

# Feature list (use more features if needed)
features = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
            'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']

entries = {}

# Heading
heading = tk.Label(center_frame, text="Breast Cancer Prediction", font=("Helvetica", 18, "bold"),
                   bg="black", fg="white")
heading.grid(row=0, column=0, columnspan=2, pady=20)

# Input fields
for i, feature in enumerate(features):
    label = tk.Label(center_frame, text=feature + ":", anchor='e', bg="black", fg="white",
                     font=("Helvetica", 11))
    label.grid(row=i+1, column=0, padx=10, pady=5, sticky="e")

    entry = tk.Entry(center_frame, width=30, bg="#222222", fg="white", insertbackground="white")
    entry.grid(row=i+1, column=1, padx=10, pady=5)
    entries[feature] = entry

# Prediction function
def predict():
    try:
        input_data = [float(entries[feature].get()) for feature in features]
        input_data = np.array(input_data).reshape(1, -1)
        input_scaled = scaler.transform(input_data)
        result = model.predict(input_scaled)

        if result[0] == 1:
            messagebox.showinfo("Prediction Result", "⚠️ Malignant (Cancer Detected)")
        else:
            messagebox.showinfo("Prediction Result", "✅ Benign (No Cancer Detected)")
    except Exception as e:
        messagebox.showerror("Input Error", f"Please enter valid numbers.\n\n{e}")

# Predict Button
predict_btn = tk.Button(center_frame, text="Predict", command=predict,
                        bg="white", fg="black", font=("Helvetica", 12, "bold"),
                        padx=10, pady=5)
predict_btn.grid(row=len(features)+2, column=0, columnspan=2, pady=30)

# Run the app
root.mainloop()
